# Model Comparison for Cryptocurrency Forecasting

This notebook compares different deep learning models for cryptocurrency price prediction.

In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import our modules
from data.data_loader import CryptoDataLoader
from data.feature_engineering import FeatureEngineer
from data.preprocessor import DataPreprocessor
from models.lstm_attention import LSTMAttention, GRUAttention
from models.transformer import TransformerModel, InformerModel
from models.ensemble import EnsembleModel
from training.trainer import ModelTrainer
from evaluation.metrics import CryptoModelEvaluator
from evaluation.visualizer import ModelVisualizer
from utils.helpers import set_seed

## 1. Setup and Configuration

In [ ]:
# Set random seed for reproducibility
set_seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Configuration
config = {
    'ticker': 'BTC-USD',
    'sequence_length': 30,
    'prediction_horizon': 1,
    'train_ratio': 0.7,
    'val_ratio': 0.15,
    'batch_size': 32,
    'epochs': 20,  # Reduced for notebook demonstration
    'lr': 0.001,
    'patience': 5
}

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 2. Data Preparation

In [ ]:
# Load and prepare data
print("Loading and preparing data...")

# Initialize components
loader = CryptoDataLoader('../data/raw')
feature_engineer = FeatureEngineer()
preprocessor = DataPreprocessor()

# Load data
data = loader.get_latest_data(config['ticker'], days=365)
print(f"Loaded {len(data)} records for {config['ticker']}")

# Feature engineering
data_with_features = feature_engineer.add_technical_indicators(data)
print(f"Added {len(feature_engineer.feature_columns)} features")

# Select top features
selected_features = feature_engineer.select_features(
    data_with_features, 
    method='correlation', 
    top_k=20
)
print(f"Selected {len(selected_features)} features")

# Create sequences
X, y = feature_engineer.create_sequences(
    data_with_features, 
    config['sequence_length'],
    config['prediction_horizon']
)
print(f"Created sequences: X shape={X.shape}, y shape={y.shape}")

# Split data
X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.split_data(
    X, y, 
    config['train_ratio'], 
    config['val_ratio']
)
print(f"Data split: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")

# Scale data
X_train_scaled, X_val_scaled, X_test_scaled, y_train_scaled, y_val_scaled, y_test_scaled = preprocessor.scale_data(
    X_train, X_val, X_test, y_train, y_val, y_test
)
print("Data scaled successfully")

# Create data loaders
train_dataset = TensorDataset(
    torch.FloatTensor(X_train_scaled), 
    torch.FloatTensor(y_train_scaled)
)
val_dataset = TensorDataset(
    torch.FloatTensor(X_val_scaled), 
    torch.FloatTensor(y_val_scaled)
)
test_dataset = TensorDataset(
    torch.FloatTensor(X_test_scaled), 
    torch.FloatTensor(y_test_scaled)
)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

print("Data loaders created")

## 3. Model Definitions

In [ ]:
# Define models to compare
input_size = len(selected_features)
models = {
    'LSTM': LSTMAttention(
        input_size=input_size,
        hidden_size=64,
        num_layers=2,
        output_size=1,
        dropout=0.2,
        bidirectional=True,
        attention_heads=4
    ),
    'GRU': GRUAttention(
        input_size=input_size,
        hidden_size=64,
        num_layers=2,
        output_size=1,
        dropout=0.2,
        bidirectional=True,
        attention_heads=4
    ),
    'Transformer': TransformerModel(
        input_size=input_size,
        d_model=64,
        nhead=4,
        num_encoder_layers=2,
        dim_feedforward=128,
        output_size=1,
        dropout=0.1
    ),
    'Informer': InformerModel(
        input_size=input_size,
        d_model=64,
        nhead=4,
        num_encoder_layers=2,
        num_decoder_layers=1,
        dim_feedforward=128,
        output_size=1,
        dropout=0.1
    )
}

# Display model information
print("Model Information:")
for name, model in models.items():
    info = model.get_model_info()
    print(f"\n{name}:")
    print(f"  Total parameters: {info['total_parameters']:,}")
    print(f"  Trainable parameters: {info['trainable_parameters']:,}")

## 4. Train Models

In [ ]:
# Train all models
model_results = {}
model_predictions = {}
model_histories = {}

for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Training {name} Model")
    print(f"{'='*50}")
    
    # Create trainer
    trainer = ModelTrainer(model, device)
    
    # Train model
    history = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=config['epochs'],
        lr=config['lr'],
        patience=config['patience'],
        early_stopping=True
    )
    
    # Evaluate model
    metrics, predictions, actuals = trainer.evaluate(test_loader, preprocessor.target_scaler)
    
    # Store results
    model_results[name] = metrics
    model_predictions[name] = predictions
    model_histories[name] = history
    
    print(f"\n{name} Results:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

## 5. Compare Model Performance

In [ ]:
# Create comparison DataFrame
results_df = pd.DataFrame(model_results).T
print("Model Comparison Results:")
pd.set_option('display.float_format', '{:.4f}'.format)
display(results_df)

# Find best model for each metric
print("\nBest Models by Metric:")
for metric in results_df.columns:
    if metric in ['MAE', 'MSE', 'RMSE', 'MAPE', 'MASE', 'Max Drawdown', 'Error Volatility']:
        best_model = results_df[metric].idxmin()
    else:
        best_model = results_df[metric].idxmax()
    print(f"  {metric}: {best_model} ({results_df[metric][best_model]:.4f})")

In [ ]:
# Plot model comparison
visualizer = ModelVisualizer()
visualizer.plot_model_comparison(
    model_results, 
    metrics=['MAE', 'RMSE', 'MAPE', 'Directional Accuracy', 'Sharpe Ratio'],
    save_path='../results/model_comparison.png'
)

## 6. Visualize Predictions

In [ ]:
# Plot predictions for all models
plt.figure(figsize=(15, 8))

# Plot actual values
n_points = min(100, len(actuals))
x_axis = range(n_points)
plt.plot(x_axis, actuals[:n_points], label='Actual', color='black', linewidth=3)

# Plot predictions for each model
colors = ['blue', 'red', 'green', 'orange', 'purple']
for i, (name, predictions) in enumerate(model_predictions.items()):
    plt.plot(x_axis, predictions[:n_points], label=name, 
             color=colors[i % len(colors)], linewidth=2, linestyle='--')

plt.title('Model Predictions Comparison', fontsize=16)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot prediction errors
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, (name, predictions) in enumerate(model_predictions.items()):
    if i < len(axes):
        errors = actuals - predictions
        axes[i].hist(errors, bins=30, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'{name} Prediction Errors')
        axes[i].set_xlabel('Error')
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)
        
        # Add vertical line at zero
        axes[i].axvline(x=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 7. Training History Comparison

In [ ]:
# Plot training histories
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

# Plot training loss
for name, history in model_histories.items():
    axes[0].plot(history['train_loss'], label=f'{name} Train')
    axes[1].plot(history['val_loss'], label=f'{name} Val')

axes[0].set_title('Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot learning rate
for name, history in model_histories.items():
    if 'learning_rate' in history:
        axes[2].plot(history['learning_rate'], label=name)

axes[2].set_title('Learning Rate')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Learning Rate')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Plot epoch time
for name, history in model_histories.items():
    if 'epoch_time' in history:
        axes[3].plot(history['epoch_time'], label=name)

axes[3].set_title('Epoch Time')
axes[3].set_xlabel('Epoch')
axes[3].set_ylabel('Time (seconds)')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Create Ensemble Model

In [ ]:
# Create ensemble model
base_models = list(models.values())
ensemble = EnsembleModel(
    models=base_models,
    input_size=input_size,
    output_size=1,
    aggregation_method='weighted_average'
)

print("Ensemble Model Information:")
ensemble_info = ensemble.get_model_info()
for key, value in ensemble_info.items():
    print(f"  {key}: {value}")

# Train ensemble
print("\nTraining Ensemble Model...")
ensemble_trainer = ModelTrainer(ensemble, device)

ensemble_history = ensemble_trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=config['epochs'],
    lr=config['lr'],
    patience=config['patience'],
    early_stopping=True
)

# Evaluate ensemble
ensemble_metrics, ensemble_predictions, ensemble_actuals = ensemble_trainer.evaluate(
    test_loader, preprocessor.target_scaler
)

print("\nEnsemble Results:")
for metric, value in ensemble_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Add ensemble to results
model_results['Ensemble'] = ensemble_metrics
model_predictions['Ensemble'] = ensemble_predictions

## 9. Final Comparison with Ensemble

In [ ]:
# Updated comparison with ensemble
final_results_df = pd.DataFrame(model_results).T
print("Final Model Comparison (with Ensemble):")
display(final_results_df)

# Find best model for each metric
print("\nBest Models by Metric (including Ensemble):")
for metric in final_results_df.columns:
    if metric in ['MAE', 'MSE', 'RMSE', 'MAPE', 'MASE', 'Max Drawdown', 'Error Volatility']:
        best_model = final_results_df[metric].idxmin()
    else:
        best_model = final_results_df[metric].idxmax()
    print(f"  {metric}: {best_model} ({final_results_df[metric][best_model]:.4f})")

In [ ]:
# Plot final comparison
visualizer.plot_model_comparison(
    model_results, 
    metrics=['MAE', 'RMSE', 'MAPE', 'Directional Accuracy', 'Sharpe Ratio'],
    save_path='../results/final_model_comparison.png'
)

In [ ]:
# Plot ensemble vs best individual model
best_individual_model = final_results_df['RMSE'].idxmin()
print(f"Comparing Ensemble with best individual model: {best_individual_model}")

plt.figure(figsize=(15, 8))

# Plot actual values
n_points = min(100, len(ensemble_actuals))
x_axis = range(n_points)
plt.plot(x_axis, ensemble_actuals[:n_points], label='Actual', color='black', linewidth=3)

# Plot best individual model
plt.plot(x_axis, model_predictions[best_individual_model][:n_points], 
         label=best_individual_model, color='blue', linewidth=2, linestyle='--')

# Plot ensemble
plt.plot(x_axis, ensemble_predictions[:n_points], 
         label='Ensemble', color='red', linewidth=2, linestyle='-')

plt.title('Ensemble vs Best Individual Model', fontsize=16)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Model Performance Summary

In [ ]:
# Create performance summary
summary_stats = pd.DataFrame()

for name, metrics in model_results.items():
    summary_stats[name] = pd.Series({
        'RMSE': metrics['RMSE'],
        'MAE': metrics['MAE'],
        'MAPE': metrics['MAPE'],
        'Directional Accuracy': metrics['Directional Accuracy'],
        'Sharpe Ratio': metrics['Sharpe Ratio'],
        'Training Time (s)': sum(model_histories[name]['epoch_time']) if name in model_histories else 0
    })

# Rank models
summary_stats['RMSE_Rank'] = summary_stats['RMSE'].rank()
summary_stats['MAE_Rank'] = summary_stats['MAE'].rank()
summary_stats['Directional_Accuracy_Rank'] = summary_stats['Directional Accuracy'].rank(ascending=False)
summary_stats['Average_Rank'] = summary_stats[['RMSE_Rank', 'MAE_Rank', 'Directional_Accuracy_Rank']].mean(axis=1)

print("Model Performance Summary:")
display(summary_stats.sort_values('Average_Rank'))

# Save results
summary_stats.to_csv('../results/model_performance_summary.csv')
print("\nResults saved to '../results/model_performance_summary.csv'")

## 11. Conclusions

### Key Findings:

1. **Best Performing Model**: [Based on results]
2. **Ensemble Performance**: The ensemble model typically provides more stable predictions
3. **Training Speed**: [Fastest model] trains quickest, while [slowest model] takes longest
4. **Prediction Accuracy**: All models achieve reasonable accuracy with [best metric] being [value]
5. **Directional Accuracy**: [Best model] is most accurate at predicting price direction

### Recommendations:

1. **For Production**: Use the ensemble model for more robust predictions
2. **For Speed**: Use [fastest accurate model] for real-time applications
3. **For Research**: Experiment with different architectures and hyperparameters
4. **For Improvement**: Consider adding more features or using longer sequences

### Future Work:

1. **Hyperparameter Tuning**: Use automated tuning for each model
2. **Feature Selection**: Experiment with different feature selection methods
3. **Multi-step Prediction**: Extend to predict multiple days ahead
4. **Probabilistic Forecasting**: Add prediction intervals
5. **Online Learning**: Implement continuous learning for market adaptation